In [1]:
%load_ext autoreload 
%autoreload 2
import cv2
import json, os, cv2
from pathlib import Path
import pandas as pd 
import matplotlib.pyplot as plt 
import numpy as np
import xarray as xr
import seaborn as sns
import h5py as hf
from tqdm import tqdm
from tqdm import tqdm
import itertools 
from scipy.interpolate import interp1d
from scipy import signal
from matplotlib.backends.backend_pdf import PdfPages
from sklearn.cluster import KMeans
import matplotlib.colors as mcolors
import os, fnmatch
from scipy.spatial.distance import cdist

import sys
sys.path.insert(0, 'C:/Users/nlab/Documents/GitHub/obstacle_avoidance')

from utils.base_functions import *
from src.utils.auxiliary import flatten_series
from src.utils.path import find
from src.base import BaseInput
from plots import plot_oa

import warnings
warnings.filterwarnings('ignore')

<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject
c:\Users\nlab\anaconda3\envs\ephys0\lib\site-packages\deeplabcut\__init__.py:78: UserWarning: 
        As PyTorch is not installed, unsupervised identity learning will not be available.
        Please run `pip install torch`, or ignore this warning.
        
  warnings.warn(


In [4]:
session = plot_oa('D:/obstacle_avoidance/metadata/G8CKRT_oa_050323.json')
session.gather_session_df('obstacle')
session.cluster(6)
df = session.df

In [ ]:
session1 = plot_oa('D:/obstacle_avoidance/metadata/101222_101322_101422.json')
session1.gather_session_df()
session1.process_df()
df1 = session1.df

In [5]:
df.head()

,first_poke,second_poke,trial_timestamps,trial_vidframes,nose_x,nose_y,leftear_x,leftear_y,rightear_x,rightear_y,...,gt_obstacleBL_y_cm,gt_obstacle_cen_x,gt_obstacle_cen_x_cm,gt_obstacle_cen_y,gt_obstacle_cen_y_cm,time,obstacle_cluster,mean_gt_obstacle_cen_x_cm,mean_gt_obstacle_cen_y_cm,cluster_label
0,57307.402176,57310.646540,"[57307.413734, 57307.429926, 57307.447129, 573...","[74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 8...","[nan, nan, nan, 616.2639770507812, 609.3107910...","[nan, nan, nan, 219.03343200683594, 216.107147...","[649.117431640625, 642.1621704101562, nan, 626...","[256.2538757324219, 251.69952392578125, nan, 2...","[651.4603881835938, 643.626220703125, nan, 634...","[262.35333251953125, 248.5648651123047, nan, 2...",...,29.099578,345.911771,26.822300,284.242782,22.040433,3.216667,0,26.752311,21.445802,0.0
1,57310.646540,57312.768435,"[57310.648051, 57310.665216, 57310.681356, 573...","[[268, 269, 270, 271, 272, 273, 274, 275, 276,...","[685.233642578125, nan, nan, nan, nan, nan, na...","[271.8834228515625, nan, nan, nan, nan, nan, n...","[668.349853515625, 672.6154174804688, 672.9995...","[266.5401916503906, 267.870361328125, 267.5038...","[677.6964111328125, 679.1038208007812, 679.879...","[285.75848388671875, 285.0168151855469, 283.65...",...,29.107143,345.948805,26.825172,284.477641,22.058644,2.116667,0,26.752311,21.445802,0.0
2,57312.768435,57314.935283,"[57312.781862, 57312.798809, 57312.816192, 573...","[396, 397, 398, 399, 400, 401, 402, 403, 404, ...","[98.14295196533203, 98.16766357421875, 98.1457...","[277.06103515625, 277.373779296875, 277.318908...","[117.95055389404297, 117.12333679199219, 117.3...","[265.9244079589844, 266.379150390625, 266.3605...","[107.47877502441406, 105.98230743408203, 105.8...","[251.98460388183594, 254.7049560546875, 254.58...",...,34.865365,433.520211,33.615535,359.767108,27.896655,2.150000,1,33.779017,27.777656,4.0
3,57314.935283,57317.483340,"[57314.949171, 57314.967206, 57314.982796, 573...","[[526, 527, 528, 529, 530, 531, 532, 533, 534,...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[673.6448974609375, 674.8700561523438, 676.655...","[265.6354675292969, 266.148193359375, 266.6850...","[673.0361328125, 674.72021484375, 676.95104980...","[287.0090637207031, 287.1199035644531, 287.185...",...,34.962080,432.745865,33.555491,360.761439,27.973756,2.516667,1,33.779017,27.777656,4.0
4,57317.483340,57319.269363,"[57317.483878, 57317.500236, 57317.518195, 573...","[678, 679, 680, 681, 682, 683, 684, 685, 686, ...","[98.41161346435547, nan, nan, nan, nan, 96.831...","[281.2069091796875, nan, nan, nan, nan, 280.89...","[110.39929962158203, 107.95673370361328, 106.4...","[278.97076416015625, 277.5727233886719, 276.53...","[107.89097595214844, 106.0604248046875, 104.72...","[259.6772766113281, 259.2562561035156, 259.172...",...,34.965838,433.428709,33.608440,360.439528,27.948795,1.783333,1,33.779017,27.777656,4.0


In [ ]:
df.columns.values

In [ ]:

#for ind,row in df.iterrows():
for clusters, cluster_name in enumerate(df['obstacle_cluster'].unique()):
    x=df.loc[df['obstacle_cluster']==cluster_name]
    mean_cenx = np.mean(x['gt_obstacle_cen_x_cm'])
    mean_ceny = np.mean(x['gt_obstacle_cen_y_cm'])
  
    for ind,row in df.iterrows(): 
        if row['obstacle_cluster'] == cluster_name:
            df.at[ind,'mean_gt_obstacle_cen_x_cm'] = mean_cenx
            df.at[ind,'mean_gt_obstacle_cen_y_cm'] = mean_ceny




In [ ]:
#for ind,row in df.iterrows():
for clusters, cluster_name in enumerate(df1['obstacle_cluster'].unique()):
    x=df1.loc[df1['obstacle_cluster']==cluster_name]
    mean_cenx = np.mean(x['gt_obstacle_cen_x_cm'])
    mean_ceny = np.mean(x['gt_obstacle_cen_y_cm'])
  
    for ind,row in df1.iterrows(): 
        if row['obstacle_cluster'] == cluster_name:
            df1.at[ind,'mean_gt_obstacle_cen_x_cm'] = mean_cenx
            df1.at[ind,'mean_gt_obstacle_cen_y_cm'] = mean_ceny

In [8]:
gt_label = df['cluster_label'].astype(int).tolist()
#create obstacle_xypos array 
obstacle_xpos = np.array(df.loc[:,'gt_obstacle_cen_x_cm'])
obstacle_ypos = np.array(df.loc[:,'gt_obstacle_cen_y_cm'])
obstacle_xypos_ar = np.stack((obstacle_xpos, obstacle_ypos))
plt.figure()
for i in range(len(df['obstacle_cluster'])):
    obstacle_xypos = obstacle_xypos_ar[:,i] 
    plt.plot(obstacle_xypos[0], obstacle_xypos[1], '*', color=list(mcolors.TABLEAU_COLORS)[gt_label[i]])
    plt.ylim([52,0]); plt.xlim([0, 72])

TypeError: list indices must be integers or slices, not float

<Figure size 640x480 with 0 Axes>

In [ ]:
plt.subplots(3,3, figsize=(25,21))


for clusters, cluster_name in enumerate(df1['obstacle_cluster'].unique()):
    x=df1.loc[df1['obstacle_cluster']==cluster_name]
    plt.subplot(3,3,cluster_name+1)
    plt.gca().set_title(cluster_name)
    plt.hist2d(x['gt_obstacle_cen_x_cm'],x['gt_obstacle_cen_y_cm'])
    #plt.ylim([52,0]); plt.xlim([0, 72])

    

In [ ]:
x['mean_gt_obstacle_cen_x_cm'].unique()

In [ ]:
len(df1)

In [ ]:
plt.subplots(3,3, figsize=(25,21))


for clusters, cluster_name in enumerate(df1['obstacle_cluster'].unique()):
    x=df1.loc[df1['obstacle_cluster']==cluster_name]
    plt.subplot(3,3,cluster_name+1)
    plt.gca().set_title(str(cluster_name)+'_' + str(len(x)))
    cluster_center = np.column_stack((x['mean_gt_obstacle_cen_x_cm'].unique(),x['mean_gt_obstacle_cen_y_cm'].unique()))
    cluster = np.column_stack((x['gt_obstacle_cen_x_cm'].to_numpy(),x['gt_obstacle_cen_y_cm'].to_numpy()))
    distances = cdist(cluster_center,cluster)
    plt.plot(x['gt_obstacle_cen_x_cm'],x['gt_obstacle_cen_y_cm'],'x')
    plt.plot(x['gt_obstacle_cen_x_cm'].iloc[np.where(distances<np.percentile(distances,95))[1]],x['gt_obstacle_cen_y_cm'].iloc[np.where(distances<np.percentile(distances,95))[1]],'.',color='black')
    plt.plot(x['mean_gt_obstacle_cen_x_cm'].unique(),x['mean_gt_obstacle_cen_y_cm'].unique(),'*',color = 'red')
    plt.ylim([52,0]); plt.xlim([0, 72])

In [6]:
for clusters, cluster_name in enumerate(df['obstacle_cluster'].unique()):
    x=df.loc[df['obstacle_cluster']==cluster_name]
    cen_x_list = x['gt_obstacle_cen_x_cm'].sort_values()
    cen_x_list = cen_x_list.to_numpy()
    cen_y_list = x['gt_obstacle_cen_y_cm'].sort_values()
    cen_y_list = cen_y_list.to_numpy()
    cen_x_smallest,cen_x_largest,cen_y_smallest,cen_y_largest = cen_x_list[0],cen_x_list[-1],cen_y_list[0],cen_y_list[-1]
    print(cluster_name,cen_x_smallest,cen_x_largest,cen_y_smallest,cen_y_largest,x['mean_gt_obstacle_cen_x_cm'].unique(),x['mean_gt_obstacle_cen_y_cm'].unique())

0 26.325371139393813 27.136299733431127 20.741230034681003 22.058644182528184 [26.75231053] [21.44580177]
1 33.160978801684045 35.07495475209891 26.95926374476633 28.035156659373722 [33.7790173] [27.77765561]
4 23.81847296075221 27.788483670995408 15.381900243792483 16.326707529009454 [26.42889006] [15.77213121]
3 24.17877996916725 26.863215561849103 27.48561630082473 28.01742278586136 [25.60993259] [27.8602421]
5 33.235051355648295 34.453496202015685 21.22587817855072 21.849141580391397 [34.10278153] [21.53113798]
2 32.06583516076989 34.997202461859246 15.114272937318955 15.922471694589127 [33.9870879] [15.64928596]


In [ ]:
## get the geometric (top,left) postion of clusters 
# Find the outliers in each clusters x,y postion
def label_cluster(df):

In [ ]:
df1['cluster_label'] = np.nan
for clusters, cluster_name in enumerate(df1['obstacle_cluster'].unique()):
    x=df1.loc[df1['obstacle_cluster']==cluster_name]
    cen_x_list = x['gt_obstacle_cen_x_cm'].sort_values()
    cen_x_list = cen_x_list.to_numpy()
    cen_y_list = x['gt_obstacle_cen_y_cm'].sort_values()
    cen_y_list = cen_y_list.to_numpy()
    cen_x_smallest,cen_x_largest,cen_y_smallest,cen_y_largest = cen_x_list[0],cen_x_list[-1],cen_y_list[0],cen_y_list[-1]
    #label cluster by obstacle post
    for ind,row in x.iterrows():
        # position top left label 0
        if 26.511267901536936 <= x['mean_gt_obstacle_cen_x_cm'].unique() <= 29.262555270323688 and 19.603003146893094 <= x['mean_gt_obstacle_cen_y_cm'].unique() <= 21.48005617413041:
            df1.at[ind,'cluster_label'] = 0
        # postion top middle lable 1     
        if 34.224618031842425  <= x['mean_gt_obstacle_cen_x_cm'].unique() <= 40.737819614353576  and 19.171765596431882  <= x['mean_gt_obstacle_cen_y_cm'].unique() <= 21.765871122032685:
            df1.at[ind,'cluster_label'] = 1
        # postion top middle lable 2     
        if 47.10153827545068   <= x['mean_gt_obstacle_cen_x_cm'].unique() <= 51.15598067362944  and 19.386902039003104  <= x['mean_gt_obstacle_cen_y_cm'].unique() <= 21.36299006442639:
            df1.at[ind,'cluster_label'] = 2
        # postion top middle lable 3     
        if 24.622638543067332    <= x['mean_gt_obstacle_cen_x_cm'].unique() <= 30.32366857885396   and 24.072987056842724  <= x['mean_gt_obstacle_cen_y_cm'].unique() <= 28.06565031743722:
            df1.at[ind,'cluster_label'] = 3 
        # postion top middle lable 4     
        if 34.33696422611072     <= x['mean_gt_obstacle_cen_x_cm'].unique() <= 39.16315460138148   and 23.33051936216178  <= x['mean_gt_obstacle_cen_y_cm'].unique() <= 28.21568031983405:
            df1.at[ind,'cluster_label'] = 4
        # postion top middle lable 5     
        if 46.78662583258851     <= x['mean_gt_obstacle_cen_x_cm'].unique() <= 51.45490492362184   and 23.30309610363196  <= x['mean_gt_obstacle_cen_y_cm'].unique() <= 26.340749658133035:
            df1.at[ind,'cluster_label'] = 5
        # postion top middle lable 6     
        if 23.672283873362495      <= x['mean_gt_obstacle_cen_x_cm'].unique() <= 29.24383212681531  and 31.13010098995012  <= x['mean_gt_obstacle_cen_y_cm'].unique() <= 32.9010213861951:
            df1.at[ind,'cluster_label'] = 6
        # postion top middle lable 7     
        if 34.99528058841633      <= x['mean_gt_obstacle_cen_x_cm'].unique() <= 38.68741237003255   and 30.937326612118863  <= x['mean_gt_obstacle_cen_y_cm'].unique() <= 32.92257553927523:
            df1.at[ind,'cluster_label'] = 7
        # postion top middle lable 8     
        if 47.0371762841755 <= x['mean_gt_obstacle_cen_x_cm'].unique() <= 52.09703145249776    and 30.463949607263373  <= x['mean_gt_obstacle_cen_y_cm'].unique() <= 32.749148951731954:
            df1.at[ind,'cluster_label'] = 8

        
                  
            

In [ ]:
print(x['mean_gt_obstacle_cen_x_cm'].unique(),x['mean_gt_obstacle_cen_y_cm'].unique())

In [ ]:
df1['cluster_label'].unique()

In [ ]:
30.463949607263373  <= x['mean_gt_obstacle_cen_y_cm'].unique() <= 32.749148951731954

In [ ]:
47.0371762841755 <= x['mean_gt_obstacle_cen_x_cm'].unique() <= 52.09703145249776 

In [ ]:
int(7)

In [ ]:
int(cluster_name)+1

In [ ]:
plt.subplots(3,3, figsize=(25,21))


for clusters, cluster_name in enumerate(df1['cluster_label'].unique()):
    x=df1.loc[df1['cluster_label']==cluster_name]
    for i, row in x.iterrows():
        if row['odd'] == True:
            plt.subplot(3,3,int(cluster_name)+1)
            plt.gca().set_aspect('equal', adjustable='box')
            plt.gca().set_title(str(row['odd'])+str(row['cluster_label']))
    
            plt.plot([row['arenaTL_x_cm'], row['arenaTR_x_cm'], row['arenaBR_x_cm'], row['arenaBL_x_cm'],row['arenaTL_x_cm']],
                [row['arenaTL_y_cm'], row['arenaTR_y_cm'], row['arenaBR_y_cm'], row['arenaBL_y_cm'],row['arenaTL_y_cm']],color='orange')


            plt.plot([row['gt_obstacleTL_x_cm'], row['gt_obstacleTR_x_cm'], row['gt_obstacleBR_x_cm'], row['gt_obstacleBL_x_cm'],row['gt_obstacleTL_x_cm']],
                [row['gt_obstacleTL_y_cm'], row['gt_obstacleTR_y_cm'], row['gt_obstacleBR_y_cm'], row['gt_obstacleBL_y_cm'],row['gt_obstacleTL_y_cm']],color='green')
    
   
    
    
            plt.scatter(row['gt_obstacle_cen_x_cm'],row['gt_obstacle_cen_y_cm'],color='blue')
            plt.scatter(row['leftportT_x_cm'],row['leftportT_y_cm'],color='blue')
            plt.scatter(row['rightportT_x_cm'],row['rightportT_y_cm'],color='black')
            sns.scatterplot(x=row['ts_nose_x_cm'],y=row['ts_nose_y_cm'],hue = enumerate(row['ts_nose_x_cm']), palette ='magma',legend=False) 
            #plt.scatter(row['wobstacle_x_cm'], row['wobstacle_y_cm'], c = list(mcolors.TABLEAU_COLORS)[ row['obstacle_cluster']])
            plt.ylim([52,0]); plt.xlim([0, 72])

In [ ]:
plt.subplots(3,3, figsize=(25,21))


for clusters, cluster_name in enumerate(df['cluster_label'].unique()):
    x=df.loc[df['cluster_label']==cluster_name]
    for i, row in x.iterrows():
        if row['odd'] == True:
            plt.subplot(3,3,int(cluster_name)+1)
            plt.gca().set_aspect('equal', adjustable='box')
            plt.gca().set_title(str(row['odd'])+str(row['cluster_label']))
    
            plt.plot([row['arenaTL_x_cm'], row['arenaTR_x_cm'], row['arenaBR_x_cm'], row['arenaBL_x_cm'],row['arenaTL_x_cm']],
                [row['arenaTL_y_cm'], row['arenaTR_y_cm'], row['arenaBR_y_cm'], row['arenaBL_y_cm'],row['arenaTL_y_cm']],color='orange')


            plt.plot([row['gt_obstacleTL_x_cm'], row['gt_obstacleTR_x_cm'], row['gt_obstacleBR_x_cm'], row['gt_obstacleBL_x_cm'],row['gt_obstacleTL_x_cm']],
                [row['gt_obstacleTL_y_cm'], row['gt_obstacleTR_y_cm'], row['gt_obstacleBR_y_cm'], row['gt_obstacleBL_y_cm'],row['gt_obstacleTL_y_cm']],color='green')
    
   
    
    
            plt.scatter(row['gt_obstacle_cen_x_cm'],row['gt_obstacle_cen_y_cm'],color='blue')
            plt.scatter(row['leftportT_x_cm'],row['leftportT_y_cm'],color='blue')
            plt.scatter(row['rightportT_x_cm'],row['rightportT_y_cm'],color='black')
            sns.scatterplot(x=row['ts_nose_x_cm'],y=row['ts_nose_y_cm'],hue = enumerate(row['ts_nose_x_cm']), palette ='magma',legend=False) 
            #plt.scatter(row['wobstacle_x_cm'], row['wobstacle_y_cm'], c = list(mcolors.TABLEAU_COLORS)[ row['obstacle_cluster']])
            plt.ylim([52,0]); plt.xlim([0, 72])

In [ ]:
df1.to_hdf('D:/obstacle_avoidance/recordings/data_h5/101222_101322_101422.h5','w')


In [ ]:
df.groupby(['animal']).count()

In [ ]:
test = pd.read_hdf(r'D:\obstacle_avoidance\recordings\101422\J19RT\oa\df_J19RT_101422_oa.h5')

In [ ]:
test['odd'] = np.nan
for i, ind in enumerate(test.index.values):
    if ind%2 == 0: # odd values
        test.at[ind, 'odd'] = True
    elif ind%2 == 1:
        test.at[ind, 'odd'] = False

In [ ]:
dist_to_posts = np.median(self.data['arenaTR_x'].iloc[0],0) - np.median(self.data['arenaTL_x'].iloc[0],0)
pxls2cm = dist_to_posts/self.dist_across_arena
self.convert_pxls_to_dist()

In [ ]:
keys = ['nose','leftear','rightear','spine','midspine','tailbase']
keys_list = list_columns(test,keys)
keys_list= [col for col in keys_list if 'likelihood' not in col]
keys_list= [col for col in keys_list if 'lind' not in col]
# check if odd or even trial
#  get first index when nose crosses a distance thresh hold
#trail start = ts
##odd tiral at 16 cm even at 56 cm     
for ind, row in test.iterrows(): 
    if row['odd'] == True:
        nose_list = row['nose_x_cm'] 
        odd_ind = np.argmax(nose_list>16)
        for key in keys_list:
            test.at[ind,'ts_' + key] = row[key][odd_ind:]
    else: 
        nose_list = row['nose_x_cm']
        even_ind = np.argmax(nose_list<56)
        for key in keys_list:
            test.at[ind,'ts_' + key] = row[key][even_ind:]

In [ ]:
np.nan

In [ ]:
obstacle_cols = list_columns(test,['obstacle'])
obstacle_cols = [col for col in obstacle_cols if 'likelihood' not in col]
for ind, row in test.iterrows():
    nose_list = row['nose_x_cm']
    middle_time = np.where((nose_list > 30) & (nose_list < 50))
    if len(middle_time[0]) == 0:
        test = test.drop(ind)
        #for col in obstacle_cols:
            #test.at[ind,'gt_'+ col]= np.nan
    else:
        first,last = [middle_time[0][i] for i in (0, -1)] 
    # calculate median of each corner
        for col in obstacle_cols:
            test.at[ind,'gt_'+ col]= np.nanmedian(row[col][first:last])


In [ ]:
for ind, row in test.iterrows():
    xvals = np.stack([row['gt_obstacleTL_x'], row['gt_obstacleTR_x'], row['gt_obstacleBL_x'], row['gt_obstacleBR_x']])
    xvals_cm = np.stack([row['gt_obstacleTL_x_cm'], row['gt_obstacleTR_x_cm'], row['gt_obstacleBL_x_cm'], row['gt_obstacleBR_x_cm']])
    test.at[ind,'gt_obstacle_cen_x' ] = np.mean(xvals)
    test.at[ind,'gt_obstacle_cen_x_cm' ] = np.mean(xvals_cm)
    yvals = np.stack([row['gt_obstacleTL_y'], row['gt_obstacleTR_y'], row['gt_obstacleBL_y'], row['gt_obstacleBR_y']])
    yvals_cm = np.stack([row['gt_obstacleTL_y_cm'], row['gt_obstacleTR_y_cm'], row['gt_obstacleBL_y_cm'], row['gt_obstacleBR_y_cm']])
    test.at[ind,'gt_obstacle_cen_y' ] = np.mean(yvals)
    test.at[ind,'gt_obstacle_cen_y_cm' ] = np.mean(yvals_cm)

In [ ]:
test

In [ ]:
middle_time

In [ ]:
len(middle_time[0])

In [ ]:
np.where((nose_list > 30) & (nose_list < 50))

In [ ]:
test.at[1,'nose_x_cm']

In [ ]:
test.at[7,'nose_x_cm']

In [ ]:
np.where((nose_list > 30) & (nose_list < 50))

In [ ]:
np.where((nose_list > 30) & (nose_list < 50))

In [ ]:
first,last = [middle_time[0][i] for i in (0, -1)] 

In [ ]:
first,last 

In [ ]:
test

In [ ]:
nose_list = test_row['nose_x_cm']
middle_time = np.where((nose_list > 30) & (nose_list < 50))
first,last = [middle_time[0][i] for i in (0, -1)]

In [ ]:
np.nanmedian(test_row[col][first:last])

In [ ]:
ind

In [ ]:
row['obstacleTL_x']

In [ ]:
test